In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix 


In [39]:
df = pd.read_csv('../resources/csgo_round_snapshots.csv')

In [40]:
df

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,de_train,True,200.0,242.0,195.0,359.0,100.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,T
122406,174.93,11.0,15.0,de_train,False,500.0,500.0,95.0,175.0,11500.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T
122407,114.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,4.0,3.0,5.0,1.0,0.0,0.0,5.0,0.0,0.0,T
122408,94.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,5.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,T


In [41]:
np.sum(df.isnull())

time_left                    0
ct_score                     0
t_score                      0
map                          0
bomb_planted                 0
                            ..
ct_grenade_molotovgrenade    0
t_grenade_molotovgrenade     0
ct_grenade_decoygrenade      0
t_grenade_decoygrenade       0
round_winner                 0
Length: 97, dtype: int64

In [42]:
rows, columns = df.shape
print(f'Rows: {rows}, Columns: {columns}')

Rows: 122410, Columns: 97


In [43]:
non_numeric_cols = df.select_dtypes(exclude=[np.number])
non_numeric_cols

,map,bomb_planted,round_winner
0,de_dust2,False,CT
1,de_dust2,False,CT
2,de_dust2,False,CT
3,de_dust2,False,CT
4,de_dust2,False,CT
...,...,...,...
122405,de_train,True,T
122406,de_train,False,T
122407,de_train,False,T
122408,de_train,False,T


In [44]:
#This cell will transform all the non-numeric columns into numeric columns
le = LabelEncoder()

df['bomb_planted'] = df['bomb_planted'].astype(int)
df = pd.get_dummies(df, columns=['map'], dtype=int)

df["round_winner"] = le.fit_transform(df["round_winner"])
label_mapping = dict(zip(le.classes_, range(len(le.classes_))))
print(label_mapping)

{'CT': 0, 'T': 1}


In [45]:
#This cell split target and the values and then split the data into training and testing, then scales the data.
X,y = df.drop(['round_winner'], axis=1), df['round_winner']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [46]:
#This cell creates a new instance of a round that the model can predict
data = {
  "time_left": 20,
  "ct_score": 5,
  "t_score": 5,
  "bomb_planted": 1,
  "ct_health": 460,
  "t_health": 300,
  "ct_armor": 400,
  "t_armor": 300,
  "ct_money": 4000,
  "t_money": 1000,
  "ct_helmets": 4,
  "t_helmets": 3,
  "ct_defuse_kits": 3,
  "ct_players_alive": 5,
  "t_players_alive": 3,
  "ct_weapon_ak47": 2,
  "t_weapon_ak47": 3,
  "ct_weapon_aug": 1,
  "t_weapon_aug": 0,
  "ct_weapon_awp": 1,
  "t_weapon_awp": 0,
  "ct_weapon_bizon": 0,
  "t_weapon_bizon": 0,
  "ct_weapon_cz75auto": 0,
  "t_weapon_cz75auto": 0,
  "ct_weapon_elite": 0,
  "t_weapon_elite": 0,
  "ct_weapon_famas": 0,
  "t_weapon_famas": 0,
  "ct_weapon_g3sg1": 0,
  "t_weapon_g3sg1": 0,
  "ct_weapon_galilar": 0,
  "t_weapon_galilar": 0,
  "ct_weapon_glock": 0,
  "t_weapon_glock": 2,
  "ct_weapon_m249": 0,
  "t_weapon_m249": 0,
  "ct_weapon_m4a1s": 1,
  "t_weapon_m4a1s": 0,
  "ct_weapon_m4a4": 0,
  "t_weapon_m4a4": 0,
  "ct_weapon_mac10": 0,
  "t_weapon_mac10": 0,
  "ct_weapon_mag7": 0,
  "t_weapon_mag7": 0,
  "ct_weapon_mp5sd": 0,
  "t_weapon_mp5sd": 0,
  "ct_weapon_mp7": 0,
  "t_weapon_mp7": 0,
  "ct_weapon_mp9": 0,
  "t_weapon_mp9": 0,
  "ct_weapon_negev": 0,
  "t_weapon_negev": 0,
  "ct_weapon_nova": 0,
  "t_weapon_nova": 0,
  "ct_weapon_p90": 0,
  "t_weapon_p90": 0,
  "ct_weapon_r8revolver": 0,
  "t_weapon_r8revolver": 0,
  "ct_weapon_sawedoff": 0,
  "t_weapon_sawedoff": 0,
  "ct_weapon_scar20": 0,
  "t_weapon_scar20": 0,
  "ct_weapon_sg553": 0,
  "t_weapon_sg553": 0,
  "ct_weapon_ssg08": 0,
  "t_weapon_ssg08": 0,
  "ct_weapon_ump45": 0,
  "t_weapon_ump45": 0,
  "ct_weapon_xm1014": 0,
  "t_weapon_xm1014": 0,
  "ct_weapon_deagle": 0,
  "t_weapon_deagle": 0,
  "ct_weapon_fiveseven": 0,
  "t_weapon_fiveseven": 0,
  "ct_weapon_usps": 5,
  "t_weapon_usps": 0,
  "ct_weapon_p250": 0,
  "t_weapon_p250": 0,
  "ct_weapon_p2000": 0,
  "t_weapon_p2000": 0,
  "ct_weapon_tec9": 0,
  "t_weapon_tec9": 0,
  "ct_grenade_hegrenade": 3,
  "t_grenade_hegrenade": 0,
  "ct_grenade_flashbang": 3,
  "t_grenade_flashbang": 0,
  "ct_grenade_smokegrenade": 3,
  "t_grenade_smokegrenade": 0,
  "ct_grenade_incendiarygrenade": 2,
  "t_grenade_incendiarygrenade": 0,
  "ct_grenade_molotovgrenade": 0,
  "t_grenade_molotovgrenade": 0,
  "ct_grenade_decoygrenade": 0,
  "t_grenade_decoygrenade": 0,
  "map_de_cache": 0,
  "map_de_dust2": 1,
  "map_de_inferno": 0,
  "map_de_mirage": 0,
  "map_de_nuke": 0,
  "map_de_overpass": 0,
  "map_de_train": 0,
  "map_de_vertigo": 0
}

phil = pd.DataFrame(data, index=[0])

phil_scaled = scaler.fit_transform(phil)

In [47]:
#This cell creates a random forest model and shows the score, prediction and the probabilities of the prediction
rf_model = RandomForestClassifier(n_jobs=4)

rf_model.fit(X_train, y_train)
rf_score = rf_model.score(X_test, y_test)
rf_predict = rf_model.predict(phil)
[rf_probabilities_ct, rf_probabilities_t] = rf_model.predict_proba(phil)[0]
rf_precision = precision_score(y_test, rf_model.predict(X_test))
rf_conf_matrix = confusion_matrix(y_test, rf_model.predict(X_test))
rf_recall = recall_score(y_test, rf_model.predict(X_test))

print("Accuracy", rf_score)
print("Precision", rf_precision)
print("Recall", rf_recall)
print("Confusion Matrix", rf_conf_matrix)
print("predict", "CT" if rf_predict == 0 else "T")
print("predict terrorist", f"{round(rf_probabilities_t * 100)}%")
print("predict counter-terrorist", f"{round(rf_probabilities_ct * 100)}%")

Accuracy 0.8792990768728045
Precision 0.8889433284337743
Recall 0.8722057527441711
Confusion Matrix [[10641  1360]
 [ 1595 10886]]
predict CT
predict terrorist 33%
predict counter-terrorist 67%


In [50]:
lr_model = LogisticRegression(max_iter=5000)

lr_model.fit(X_train, y_train)
lr_score = lr_model.score(X_test, y_test)
lr_predict = lr_model.predict(phil)
[lr_probabilities_ct, lr_probabilities_t] = lr_model.predict_proba(phil)[0]


print("score", lr_score)
print("predict", "CT" if lr_predict == 0 else "T")
print("predict terrorist", f"{round(lr_probabilities_t * 100)}%")
print("predict counter-terrorist", f"{round(lr_probabilities_ct * 100)}%")

score 0.7406257658688016
predict CT
predict terrorist 21%
predict counter-terrorist 79%


In [49]:
gb_model = GradientBoostingClassifier()

gb_model.fit(X_train, y_train)
gb_score = gb_model.score(X_test, y_test)
gb_predict = gb_model.predict(phil)
[gb_probabilities_ct, gb_probabilities_t] = gb_model.predict_proba(phil)[0]


print("score", gb_score)
print("predict", "CT" if gb_predict == 0 else "T")
print("predict terrorist", f"{round(gb_probabilities_t * 100)}%")
print("predict counter-terrorist", f"{round(gb_probabilities_ct * 100)}%")

score 0.7537374397516543
predict CT
predict terrorist 40%
predict counter-terrorist 60%
